In [9]:
from sympy import *
import pandas as pd

import re

def superscriptify(expr):
    """Convert an expression's powers (**) into unicode superscripts and leave multiplication as is."""
    if expr is None:
        return "*"
    s = str(expr)

    # Map digits and minus sign to superscripts
    superscript_map = {
        '0': '⁰', '1': '¹', '2': '²', '3': '³', '4': '⁴',
        '5': '⁵', '6': '⁶', '7': '⁷', '8': '⁸', '9': '⁹',
        '-': '⁻'
    }

    def replace_power(match):
        exponent_str = match.group(1)
        # Convert each character of the exponent to its superscript equivalent
        return ''.join(superscript_map.get(ch, ch) for ch in exponent_str)

    # Replace all occurrences of '**<integer>' with the corresponding superscript characters
    s = re.sub(r'\*\*(-?\d+)', lambda m: replace_power(m), s)

    return s

def extended_euclidean_algorithm_polys(N, M, x, beatify: bool = true):
    # Initialize variables for the extended Euclidean algorithm
    r = [N, M]  # Remainders
    s = [Poly(1, x), Poly(0, x)]  # Coefficients of N(x)
    t = [Poly(0, x), Poly(1, x)]  # Coefficients of M(x)

    # Perform the extended Euclidean algorithm
    k = 0
    while r[k+1] != 0:
        quotient, remainder = div(r[k], r[k+1])
        r.append(remainder)
        if remainder == 0:
            s.append(None)
            t.append(None)
        else:
            s.append(s[k] - quotient * s[k+1])
            t.append(t[k] - quotient * t[k+1])
        k += 1

    results = []
    for i in range(len(r)):
        if beatify:
            r_str = superscriptify(r[i].as_expr() if r[i] is not None else None)
            s_str = superscriptify(s[i].as_expr() if s[i] is not None else None)
            t_str = superscriptify(t[i].as_expr() if t[i] is not None else None)
        else:
            r_str = r[i].as_expr() if r[i] is not None else None
            s_str = s[i].as_expr() if s[i] is not None else None
            t_str = t[i].as_expr() if t[i] is not None else None
        results.append({
            "k": i,
            "r_k(x)": r_str,
            "s_k(x)": s_str,
            "t_k(x)": t_str
        })

    df = pd.DataFrame(results)
    return df

# Example usage
x = symbols('x', real=True)
N = Poly(x**4 + x**3 - 2*x**2 + 2*x - 2, x)  # N(x)
M = Poly(x**2 + 2*x - 3, x)                  # M(x)
df_results = extended_euclidean_algorithm_polys(N, M, x, true)


In [10]:
df_results.head(10)

,k,r_k(x),s_k(x),t_k(x)
0,0,x⁴ + x³ - 2*x² + 2*x - 2,1,0
1,1,x² + 2*x - 3,0,1
2,2,7 - 7*x,1,-x² + x - 3
3,3,0,*,*
